In [1]:
import networkx as nx
import numpy as np
import pandas as pd
from scipy import special
from scipy.spatial import distance
import seaborn as sns
from sklearn import metrics
import tqdm

from scripture_graph import graph_lib

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
digraph = nx.read_graphml('../../../scripture_graph.graphml')
print(digraph.number_of_nodes(), digraph.number_of_edges())
graph_lib.remove_topic_nodes(digraph)
print(digraph.number_of_nodes(), digraph.number_of_edges())
graph = digraph.to_undirected()
print(graph.number_of_nodes(), graph.number_of_edges())

48566 183072
41995 45985
41995 26946


In [3]:
# nx.jaccard_coefficient is too slow; we can roll our own with the adjacency matrix.
possible = special.comb(graph.number_of_nodes(), 2, exact=True) - graph.number_of_nodes()
print(graph.number_of_edges(), possible, graph.number_of_edges() / possible * 100)

26946 881727020 0.00305604789110353


In [4]:
a = nx.adjacency_matrix(graph)
type(a)

scipy.sparse.csr.csr_matrix

In [5]:
# What is the average number of connections?
total = a.sum(axis=1)
print(np.mean(total), np.median(total))
# now without the singletons
print(np.mean(total[total > 0]), np.median(total[total > 0]))

1.2832956304321943 [[ 0.  0.  0. ...  2. 11.  3.]]
2.7284325637910083 [[ 1.  1.  1. ...  2. 11.  3.]]


In [6]:
# scipy jaccard sets the distance to zero when there are no common neighbors
def jaccard(a):
    ab = a @ a.T
    aa = a.sum(axis=1)
    bb = aa.T
    return ab / (aa + bb - ab)

In [ ]:
%%time
s = jaccard(a)
s = np.nan_to_num(s, copy=False)
s[np.diag_indices_from(s)] = 0
print(s.shape)

In [ ]:
print(nz.shape, (nz > 0).sum())

In [ ]:
%%time
nodes = np.asarray(graph.nodes())
mask = np.where(s > 0)
rows = []
for i, j in zip(*mask):
    order = sorted([nodes[i], nodes[j]])
    a = set([n[1] for n in graph.edges(order[0])])
    b = set([n[1] for n in graph.edges(order[1])])
    rows.append({
        'a': order[0], 
        'b': order[1], 
        'jaccard': s[i, j], 
        'intersection': len(a & b), 
        'union': len(a | b),
    })
df = pd.DataFrame(rows)
print(df.shape)
df = df.drop_duplicates(['a', 'b'])
print(df.shape)
df.head()

In [ ]:
df.shape[0] / possible * 100

In [ ]:
sns.set_style('whitegrid')
fig, ax = subplots()
sns.ecdfplot(data=df, x='jaccard', ax=ax)
suptitle('Cumulative Jaccard Similarity Counts')
ax.set_xlabel('Jaccard Similarity (Excluding Zero)')
ax.set_xlim(0, 1)
fig.savefig('jaccard-cdf.png', dpi=300, bbox_inches='tight')

In [ ]:
sns.boxplot(data=df, x='intersection', y='jaccard')

In [ ]:
sns.boxplot(data=df, x='intersection', y='union')

In [ ]:
df.sort_values('intersection', ascending=False)

In [ ]:
df.sort_values('union', ascending=False)

In [ ]:
df.sort_values('jaccard', ascending=False)

In [ ]:
df[df.jaccard == 1].sort_values('intersection', ascending=False)

In [ ]:
df[df.jaccard == 1].intersection.value_counts()

In [ ]:
# singletons: no outgoing or incoming edges
num_singletons = 0
for node in graph.nodes:
    if not graph.edges(node):
        num_singletons += 1
print(num_singletons, graph.number_of_nodes(), num_singletons / graph.number_of_nodes())

In [ ]:
df[df.intersection > 1].sort_values(['intersection', 'jaccard'], ascending=False)